## 7-2 量子線形システム問題

### 7-2-1 HHL アルゴリズム
Harrow-Hassidim-Lloyd による逆行列計算アルゴリズム (HHL アルゴリズム)[1] は、$N \times N$ の疎 (sparse) 行列 $A$、$N$ 次元のベクトル $\bf{b}$ について、
$$
|\bf{x} \rangle \xrightarrow{\text{HHL}} | A^{-1}\bf{b} \rangle
$$
という変換を効率良く計算するアルゴリズムである。

エルミート行列 $A$ の固有値を $0<\lambda_i<1$、規格化された固有ベクトルを $\bf{u}_i$ とする ($A = \sum_i \lambda_i　|u_i \rangle \langle u_i|$)。この時、$U = \exp(i A t)$ はユニタリー演算子である。
もし $A$ がエルミート行列でなければ、
$$
A^{\prime} = \left(
\begin{array}{ll}
O & A \\
A^{\dagger} & O
\end{array}
\right)
$$
とする事でエルミート行列にする事ができる。以下では、 $A$ をエルミート行列であると仮定して説明を進める。
また、
* $s$ を $A$ の sparsity
* $\kappa$ を $A$ の条件数: $\kappa = \lambda_{\textrm{max}/\textrm{min}}$
* $\epsilon$ を出力状態の $| A^{-1}\bf{b} \rangle$ からの誤差

と定義する。

オリジナルバージョン[1]の HHL アルゴリズムは大まかに以下のようなものである (TODO: FIG)。

1. 入力状態 $|\bf{b} \rangle = \sum_i b_i |u_i \rangle$ を qRAM などを駆使して用意する (qRAM については後ほど説明する)。
これは、$A$ の規定状態を用いて、$|\bf{b} \rangle = \sum_i \beta_i |u_i \rangle$ と表す事ができる。

2. 制御ユニタリー演算 $\Lambda(e^{i A t})$ を用いた位相推定アルゴリズムによって 
$\sum_i \beta_i |u_i \rangle |\lambda_i \rangle$
の状態を用意する。

これを実現するために、先ず
$$
|\Psi_0 \rangle = \sqrt{\frac{2}{T}} \sum^{T-1}_{\tau=0} \sin \frac{\pi (\tau + 1/2)}{T} |\tau \rangle
$$
の状態を用意する。
ただし、$t_0 = \mathcal{O}(\kappa/\epsilon)$ とする時、$T$ は $e^{i A t}$ をシミュレートするのにかかる時間、
$T = \mathcal{O}(\log(N) s^2 t_0)$ である。

これに制御ユニタリー演算
$$
\Lambda(U) = \sum^{T-1}_{\tau=0} |\tau \rangle \langle \tau| \otimes \exp \left(i A \tau t_0 / T \right)
$$
を作用させると、

$$
|\Psi_0 \rangle |b \rangle \xrightarrow{\text{QPE}} \sqrt{\frac{2}{T}} \sum^N_j \beta_j \left[\sum^{T-1}_{\tau=0} \exp \left(\frac{i \lambda_j t_0 \tau}{T} \right) \sin \frac{\pi (\tau + 1/2)}{T} |\tau \rangle \right] |u_i \rangle
$$
の状態を得る。

これを Fourier 基底 $|k \rangle$ で表すと

$$
|\Psi_0 \rangle |b \rangle \xrightarrow{\text{QPE}} \sum^N_j \beta_j  |\tau \rangle \sum^{T-1}_{k=0} \alpha_{k|j} |k \rangle |u_i \rangle
$$

ただし、
$$
\alpha_{k|j} = \sqrt{\frac{2}{T}} \sum^{T-1}_{\tau=0} \exp \left[\frac{i \tau}{T} (\lambda_j t_0 - 2 \pi k) \right] \sin \frac{\pi (\tau + 1/2)}{T}
$$
$\alpha_{k|j}$ は $\lambda_j \approx 2 \pi k /t_0$ で強い最大値を持つので、これを基底状態 $|k \rangle$ を $\tilde{\lambda_k} = 2 \pi k /t_0$ を用いて書き直すと、

$$
\sum^N_j \beta_j \sum^{T-1}_{k=0} \alpha_{k|j} |\tilde{\lambda_k} \rangle |u_j \rangle
$$

の状態を得る。ただし、$|\tilde{\lambda_k} \rangle$ は二進数表記 ($0.j_1 j_2 \ldots j_t$)。

3. さらに補助ビットを追加し制御回転ゲート $\Lambda(R(\lambda^{-1}_i))$ をかける。
ただし、

$$
R(\theta) = \exp(-i \theta Y) = \left(
\begin{array}{ll}
\cos \theta & -\sin \theta \\
\sin \theta & \cos \theta
\end{array}
\right)
$$
ここで、$C$ を規格化定数とした時、$\cos \theta = C/\lambda_i$。

制御回転ゲートを補助ビットに作用させると
$$
\sum^N_j \beta_j \sum^{T-1}_{k=0} \alpha_{k|j} |\tilde{\lambda_k} \rangle |u_j \rangle |0 \rangle 
\xrightarrow{\Lambda(R(\lambda^{-1}_i))}
\sum^N_j \beta_j \sum^{T-1}_{k=0} \alpha_{k|j} |\tilde{\lambda_k} \rangle |u_j \rangle \left(\sqrt{1-\frac{C^2}{\lambda^2_j}}|0 \rangle + \frac{C}{\lambda_j} |1 \rangle \right)
$$
の状態を得る。

4.  逆位相推定 (uncomputing) を作用させると
$$
\sum^N_j \beta_j \sum^{T-1}_{k=0} \alpha_{k|j} |\tilde{\lambda_k} \rangle |u_j \rangle \left(\sqrt{1-\frac{C^2}{\lambda^2_j}}|0 \rangle + \frac{C}{\lambda_j} |1 \rangle \right)
\xrightarrow{\text{uncomputing}}
\sum^N_{i=j} \beta_j |u_j \rangle |0 \rangle \left(\sqrt{1-\frac{C^2}{\lambda^2_j}}|0 \rangle + \frac{C}{\lambda_j} |1 \rangle \right)
$$

5. 補助ビットを測定して $| 1 \rangle$ が観測された時、$|A^{-1} \bf{b} \rangle$ を得る。
$ A^{-1} = \sum_i \lambda^{-1}_i |u_i \rangle \langle u_i| $
である事を思い出すと、
$$
C \sum^N_{i=1} \beta_i/\lambda_i |u_i \rangle \propto \sum_i \beta_i/\lambda_i　|u_i \rangle = |A^{-1} b \rangle
$$
が得られる。

さて、現在最も効率的な HHL アルゴリズムでの計算量は、$O(s\kappa \textrm{poly} \log (s\kappa/\epsilon))$ 
である事が知られている。

一方、古典アルゴリズムでベストの共役勾配法では行列の逆変換実行時間は
$O(Ns\kappa \log(1/\epsilon))$ 
なので、行列のスパース性 ($s = \mathcal{O}(\textrm{poly} \log N)$) を仮定すれば、HHL アルゴリズムは行列の次元 $N$ について指数加速を実現している。ただし $s$ と $\kappa$ については線形に遅くなっており、$\epsilon$ については指数的に遅くなっている ($\epsilon$ に関しては Childs et. al. によって解決されている)。

また、上記の説明でも見たように、HHL アルゴリズムが適応できる量子線形システム問題には厳しい制限がついている。
例えば、入力 $\bf{b}$ が量子状態で与えられる事が必要であり、これを効率的 ($\mathcal{O}(\mathrm{poly} \log(N))$ 時間) にできなければ、アルゴリズムの指数加速が相殺されてしまう。
また、解の出力も同様に量子状態で与えらる。これから全ての成分を読み出そうとすると $\mathcal{O}(N)$ 時間が必要となり、指数加速が相殺されてしまう。

HHL アルゴリズムは $x$ のサンプリングが効果的なアルゴリズムのサブルーチンとして用いられるべきである。

## 参考文献
[1] Quantum algorithm for linear systems of equations, A. W. Harrow, A. Hassidim, and S. Lloyd, Physical review letters 103.15 (2009), p. 150502. <br>
[2] Quantum linear systems algorithms: a primer, D. Dervovic, M. Herbster, P. Mountney, S. Severini, N. Usher, L. Wossnig, preprint: arXiv:1802.08227" <br>
[3] Read the fine print
Scott Aaronson 
Nature Physics volume 11, pages291–293 (2015)